In [1]:
from jesse import jesse
jesse.SetOptions()
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pybaseball as mlb
import mlbgame
from IPython.display import clear_output

import pybaseball as _pybaseball
import preprocess
from preprocess import service, model, dates
#from process import batter_processing


/Users/downing.jesse/Projects/MLB_statTRAX/python
Batter data is current.


In [2]:
# this was build by preprocess
batter_data = pd.read_csv('../data/csv/raw_batter_stats.csv')

In [16]:
a = list(batter_data.DATE.unique())

In [18]:
a.sort()

In [19]:
a

['2016-04-03',
 '2016-04-04',
 '2016-04-05',
 '2016-04-06',
 '2016-04-07',
 '2016-04-08',
 '2016-04-09',
 '2016-04-10',
 '2016-04-11',
 '2016-04-12',
 '2016-04-13',
 '2016-04-14',
 '2016-04-15',
 '2016-04-16',
 '2016-04-17',
 '2016-04-18',
 '2016-04-19',
 '2016-04-20',
 '2016-04-21',
 '2016-04-22',
 '2016-04-23',
 '2016-04-24',
 '2016-04-25',
 '2016-04-26',
 '2016-04-27',
 '2016-04-28',
 '2016-04-29',
 '2016-04-30',
 '2016-05-01',
 '2016-05-02',
 '2016-05-03',
 '2016-05-04',
 '2016-05-05',
 '2016-05-06',
 '2016-05-07',
 '2016-05-08',
 '2016-05-09',
 '2016-05-10',
 '2016-05-11',
 '2016-05-12',
 '2016-05-13',
 '2016-05-14',
 '2016-05-15',
 '2016-05-16',
 '2016-05-17',
 '2016-05-18',
 '2016-05-19',
 '2016-05-20',
 '2016-05-21',
 '2016-05-22',
 '2016-05-23',
 '2016-05-24',
 '2016-05-25',
 '2016-05-26',
 '2016-05-27',
 '2016-05-28',
 '2016-05-29',
 '2016-05-30',
 '2016-05-31',
 '2016-06-01',
 '2016-06-02',
 '2016-06-03',
 '2016-06-04',
 '2016-06-05',
 '2016-06-06',
 '2016-06-07',
 '2016-06-

In [13]:
def get_mlb_dates(year=None):
        obj = dates.GetSchedule(year)
        obj.set_prior_years()
        obj.set_mlb_dates()
        obj.set_endpoints()
        obj.set_all_star_break_days()
        obj.set_season_days_so_far()
        return obj


In [19]:
date_obj = get_mlb_dates()
season_days_so_far = date_obj.get_season_days_so_far()

In [27]:

season_days_so_far = [ '2019-06-01',
 '2019-06-02',]
len_days = 10
ct = 0

In [43]:

def get_raw_stats(season_days_so_far, len_days, ct):
        df_lst = []
        for i in range(len(season_days_so_far)):
            ct += 1
            #print(ct, len_days)
            d = season_days_so_far[i]
            try:
                tmp_df = _pybaseball.batting_stats_range(str(d), )
                tmp_df['DATE'] = d
                pct_complete = int(round(ct/len_days * 100, 0))
                print('FIRST-TIME-USE INITIALIZE')
                print('processing... ', d)
                print("{}%".format(pct_complete))
                clear_output(wait=True)
                df_lst.append(tmp_df)
            except IndexError:
                continue
        #return pd.concat(df_lst, ignore_index=True), ct
        return df_lst, ct
        #return df_lst

In [37]:
lst, ct = get_raw_stats(season_days_so_far, len_days, ct=0)

FIRST-TIME-USE INITIALIZE
processing...  2019-06-02
20%


In [40]:
df = pd.concat(lst, ignore_index=True)

In [49]:
def set_raw_stats(seasons=None):
    if seasons:
        df_lst2 = []
        ct = 0
        len_days = 2
        for season in seasons:
            date_obj = get_mlb_dates(season)
            season_days_so_far = date_obj.get_season_days_so_far()[:2]
            lst, ct = get_raw_stats(season_days_so_far, len_days, ct)
            df_lst2.append(lst)
        df_lst2 = [df for sublist in df_lst2 for df in sublist]
        raw_stats = pd.concat(df_lst2, ignore_index=True)
        return raw_stats
    else: 
        ct = 0
        date_obj = get_mlb_dates()
        season_days_so_far = date_obj.get_season_days_so_far()
        lst, ct = get_raw_stats(season_days_so_far, len(season_days_so_far), ct)
        raw_stats = pd.concat(lst, ignore_index=True)
        return raw_stats

In [50]:
set_raw_stats([2016, 2019])

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,DATE
0,Matt Adams,27,1157,MLB-NL,St. Louis,1,4,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0.000,0.000,0.000,0.000,2016-04-03
1,Jose Bautista,35,1157,MLB-AL,Toronto,1,4,2,1,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0.000,0.500,0.000,0.500,2016-04-03
2,Asdrubal Cabrera,30,1157,MLB-NL,New York,1,4,4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.250,0.250,0.250,0.500,2016-04-03
3,Lorenzo Cain,30,1157,MLB-AL,Kansas City,1,4,2,2,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0.500,0.750,0.500,1.250,2016-04-03
4,Eric Campbell,29,1157,MLB-NL,New York,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,2016-04-03
5,Matt Carpenter,30,1157,MLB-NL,St. Louis,1,5,4,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0.250,0.400,0.250,0.650,2016-04-03
6,Francisco Cervelli,30,1157,MLB-NL,Pittsburgh,1,4,3,1,2,0,1,0,0,1,0,0,0,0,0,0,0,1,0.667,0.750,1.333,2.083,2016-04-03
7,Yoenis Cespedes,30,1157,MLB-NL,New York,1,5,4,1,1,0,0,0,0,1,0,2,0,0,0,0,0,0,0.250,0.400,0.250,0.650,2016-04-03
8,Chris Colabello,32,1157,MLB-AL,Toronto,1,4,4,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0.000,0.000,0.000,0.000,2016-04-03
9,Michael Conforto,23,1157,MLB-NL,New York,1,4,2,0,2,1,0,0,0,2,0,0,0,0,0,0,0,1,1.000,1.000,1.500,2.500,2016-04-03


### Test process

In [ ]:
build_path()

In [ ]:
read_obj = model.ReadData()
read_obj.raw_stats_df

In [ ]:
 #preprocess.get_raw_batter_stats(year=preprocess.get_prior_years())
read_obj.set_raw_stats_df() 

In [ ]:
read_obj.update_raw_stats_df()

In [ ]:
#read_obj.update_raw_stats_df()
import pybaseball as _pybaseball
#dates_we_dont_got = ['2019-05-31', '2019-06-01', '2019-06-02']
#new_df = _pd.DataFrame(preprocess.service.get_raw_stats(dates_we_dont_got))

def get_raw_stats(season_days_so_far):
        df_lst = []
        for d in season_days_so_far:
            print(d)
            try:
                tmp_df = _pybaseball.batting_stats_range(str(d), )
                tmp_df['DATE'] = d
                df_lst.append(tmp_df)
            except:
                continue
        new_df = pd.concat(df_lst)
        return new_df

In [ ]:
df = get_raw_stats(['2019-05-31', '2019-06-01', '2019-06-02'])
df.head()

In [ ]:
#dates = preprocess.get_mlb_dates()
# def get_mlb_dates(year=None):
#         obj = dates.GetSchedule(year)
#         obj.set_prior_years()
#         obj.set_mlb_dates()
#         obj.set_endpoints()
#         obj.set_all_star_break_days()
#         obj.set_season_days_so_far()
#         return obj

In [ ]:

save_obj = model.StoreData()
save_obj.save(read_obj.get_raw_stats_df())

In [ ]:
read_obj.get_raw_stats_df()

In [ ]:
# this was build by preprocess
batter_data = pd.read_csv('../data/csv/raw_batter_stats.csv')

In [ ]:
batter_data 

In [ ]:
cln_batter_obj = batter_processing.RawBatter(batter_data)
cln_batter_obj.process_batter()
cln_batter = cln_batter_obj.get_processed_batter()

In [ ]:
cln_batter.head()

In [ ]:
# Look for a file
# If it's there, list all its dates
# Pull a list of all the dates I want

In [ ]:
test.Name.value_counts()

In [ ]:
class ReadData:
    def __init__(self):
        self.raw_stats_df = None
    def set_raw_stats_df(self):
        try:
            self.raw_stats_df = _pd.read_csv('../data/csv/raw_batter_stats.csv')
            print('Pulling raw batter stats from existing file...')
        except OSError:
            print('FIRST-TIME-USE INITIALIZE: pulling batter data from pybaseball api...')
            self.raw_stats_df = preprocess.get_raw_batter_stats(year=preprocess.get_prior_years())
    def update_dates_in_raw_stats_df(self):
        dates = preprocess.get_mlb_dates()
        season_days_so_far = dates.get_season_days_so_far()
        dates_we_got = list(self.raw_stats_df.DATE)
        dates_we_dont_got = [d for d in season_days_so_far if d not in dates_we_got]
        if len(dates_we_dont_got) > 0:
            print('Updating batter data to most recent date...')
            print(dates_we_dont_got)
            new_df = _pd.DataFrame(preprocess.service.get_raw_stats(dates_we_dont_got))
            self.raw_stats_df = _pd.concat([self.raw_stats_df, new_df])
        else: print('Batter data is current.')
        self.raw_stats_df.drop_duplicates(inplace=True)
        self.raw_stats_df.sort_values('DATE', inplace=True)
    def get_raw_stats_df(self):
        return self.raw_stats_df

In [ ]:
load = pd.read_csv('../data/csv/raw_batter_stats.csv')
load.shape

In [ ]:
test.shape

### Aggregate all player stats for 2018

In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ', d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.shape

### Get key for a player

In [ ]:
mlb.playerid_lookup('gallo', 'joey')

In [ ]:
id = int(mlb.playerid_lookup('gallo', 'joey')['key_mlbam'])
print(id)

### Get season-level stats for a all batters

In [ ]:
mlb.batting_stats_bref(season=2019).head() # entire season
mlb.batting_stats_range(start_dt='2019-03-20', end_dt=today) # specified range

#data = mlb.batting_stats_range("2016-08-24",) 
data = mlb.batting_stats_range("2019-05-28", ) 


In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ',d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.sort_values(['Name', 'DATE'], inplace=True)

In [ ]:
df

In [ ]:
today = datetime.today()
days = [today - datetime.timedelta(days=x) for x in range(0, numdays)]

In [ ]:
from datetime import date, timedelta

d1 = date(2008, 8, 15)  # start date
d2 = date(2008, 9, 15)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    print(d1 + timedelta(days=i))



In [ ]:
# Make a list of all days during 2019 baseball season (from beginning to now)
# For each day, pull all data, and append to master dataframe
# data

## Temp

### Get raw stats for a batter

In [ ]:
#gallo_stats = mlb.statcast_batter('2019-04-01', '2019-05-01', id)
gallo_stats = mlb.statcast_batter(start_dt='2019-02-20', end_dt=today, player_id=id)

In [ ]:
gallo_stats[gallo_stats.game_date == '2019-05-01']

In [ ]:
gallo_stats

### Grab all batted ball events for a single day for a single player

In [ ]:
gallo_stats[(gallo_stats.game_date == '2019-05-01') & (~gallo_stats.events.isnull())]

In [ ]:
gallo_stats.events.value_counts()

## What other player sources exist?
* Lahman
** batting stats by year
** updated at the end of each season (current is through 2018)
* fangraps
* baseball reference
* baseball savants


### Do I have to parse the data to one-record-per-game format myself?

### Can I use retrosheet for past years, and use stat_cast for current year?

### How do I define the first game date of the season?

### I should save all player IDs to a table, and append any new players?
